# Introduction
The following code for NSE has been picked up from [Ewald's examples](https://github.com/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb) given below

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

from ib_insync import *
util.startLoop()

ib=IB()
ib.connect('127.0.0.1', 4004, clientId=12)

<IB connected to 127.0.0.1:4004 clientId=12>

In [118]:
# Get the list of Equity Option scrips
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1]          # It's the second table in the url

# Replace & for M&M, L&T, etc. This is not needed for IBKR
mask = paisa.loc[paisa.Symbol.str.contains("&"), "Symbol"]
paisa.loc[paisa.Symbol.isin(mask), "Symbol"] = mask.str.replace("&", "")

# Add columns
paisa["iSymbol"] =  paisa.Symbol.tolist()  # For IBKR scrip names
paisa["Exchange"] = 'NSE'   # for the Exchange

# Exclusions and Inclusions
vix_nifty = ['VIX', 'NIFTY']     # mask for Index
symbol_rename = {'NIFTYCPSE': 'CPSE', 'NIFTY': 'NIFTY50'}   # renamed for IBKR
symbol_del = ['INDIAVIX']

# Build the Index dataframe
index_mask = paisa.Symbol.str.contains('|'.join(vix_nifty))
nse_ind = paisa.loc[index_mask, :] # Keep only VIX and NIFTY

nse_ind = nse_ind.loc[~nse_ind.Symbol.str.contains('|'.join(symbol_del)), :] # INDIAVIX not in IBKR
nse_ind = nse_ind.replace({"iSymbol": symbol_rename}) # Rename scrips to align with IBKR

# Build the Index dataframe
nse_ind["secType"] = 'Index' #Identify security type as Index

# Qualify the Index dataframe
q_idx_df = list(map(lambda s, e : Index(symbol=s, exchange=e), 
                             nse_ind.iSymbol, nse_ind.Exchange))
qualIndex = ib.qualifyContracts(*q_idx_df)

# Build the Stock dataframe
nse_stk = paisa.loc[~index_mask, :].reset_index(drop=True) # Not the index mask!

# Shorten long scrip names in line with IBKR
iSymbol = nse_stk.Symbol.str.slice(0,9)
nse_stk.loc[:, "iSymbol"] = iSymbol

# Qualify the Stock dataframe
q_stk_df = list(map(lambda s, e: Stock(symbol=s, exchange=e),
                                      nse_stk.iSymbol, nse_stk.Exchange))
qualStock = ib.qualifyContracts(*q_stk_df)

In [148]:
def get_chains(scrips):
    '''Gets the option chains for the scrips passed
       Args:
          (scrips): list: qualified list of equity or index options with conId
       Returns:
          list of option chain'''
    
    # Prepare a sequence for ib.reqSecDefOptParams
    seq = [(i.symbol, '', i.secType, i.conId) for i in scrips]

    # build the chains
    chains = [ib.reqSecDefOptParams(*p) for p in seq]
    # list(map(lambda p: ib.reqSecDefOptParams(*p), seq))  # Another way of doing!
    
    return chains

chains = get_chains(qualStock[0:2])

In [260]:
# function to get days to expiry
def get_dte(Expiry):
    '''Gives the expiry date
    Arg:
       (Expiry) = str of yyyymmdd format
    Returns:
       dte = int days to expiry'''
    exp_date = datetime.strptime(Expiry, '%Y%m%d')
    dte = (exp_date- datetime.now()).days
    return dte

In [284]:
for elems in chains:
    for strikes in elems:
        strikez = list(strikes.strikes)


[130.0,
 135.0,
 140.0,
 145.0,
 150.0,
 155.0,
 160.0,
 35.0,
 37.5,
 165.0,
 40.0,
 42.5,
 45.0,
 47.5,
 50.0,
 52.5,
 55.0,
 57.5,
 60.0,
 62.5,
 65.0,
 67.5,
 70.0,
 72.5,
 75.0,
 77.5,
 80.0,
 82.5,
 85.0,
 87.5,
 90.0,
 92.5,
 95.0,
 97.5,
 100.0,
 102.5,
 105.0,
 107.5,
 110.0,
 112.5,
 115.0,
 117.5,
 120.0,
 122.5,
 125.0]

In [275]:
chains

[[OptionChain(exchange='NSE', underlyingConId=119653211, tradingClass='PCJEWELLER', multiplier='1', expirations={'20180726', '20180830', '20180927'}, strikes={260.0, 650.0, 140.0, 400.0, 20.0, 150.0, 280.0, 160.0, 550.0, 40.0, 300.0, 50.0, 180.0, 60.0, 700.0, 320.0, 450.0, 200.0, 80.0, 340.0, 600.0, 220.0, 350.0, 100.0, 750.0, 240.0, 500.0, 120.0, 250.0})],
 [OptionChain(exchange='NSE', underlyingConId=80423707, tradingClass='JISLJALEQS', multiplier='1', expirations={'20180726', '20180830', '20180927'}, strikes={130.0, 135.0, 140.0, 145.0, 150.0, 155.0, 160.0, 35.0, 37.5, 165.0, 40.0, 42.5, 45.0, 47.5, 50.0, 52.5, 55.0, 57.5, 60.0, 62.5, 65.0, 67.5, 70.0, 72.5, 75.0, 77.5, 80.0, 82.5, 85.0, 87.5, 90.0, 92.5, 95.0, 97.5, 100.0, 102.5, 105.0, 107.5, 110.0, 112.5, 115.0, 117.5, 120.0, 122.5, 125.0})]]

In [116]:
{'Class':[i.tradingClass for c in chains for i in c], 'Strikes':[i.strikes for c in chains for i in c]}

['PCJEWELLER', 'JISLJALEQS']

In [90]:
len(chains[0][0][5])

29

In [94]:
chains

[[OptionChain(exchange='NSE', underlyingConId=119653211, tradingClass='PCJEWELLER', multiplier='1', expirations={'20180726', '20180830', '20180927'}, strikes={260.0, 650.0, 140.0, 400.0, 20.0, 150.0, 280.0, 160.0, 550.0, 40.0, 300.0, 50.0, 180.0, 60.0, 700.0, 320.0, 450.0, 200.0, 80.0, 340.0, 600.0, 220.0, 350.0, 100.0, 750.0, 240.0, 500.0, 120.0, 250.0})],
 [OptionChain(exchange='NSE', underlyingConId=80423707, tradingClass='JISLJALEQS', multiplier='1', expirations={'20180726', '20180830', '20180927'}, strikes={130.0, 135.0, 140.0, 145.0, 150.0, 155.0, 160.0, 35.0, 37.5, 165.0, 40.0, 42.5, 45.0, 47.5, 50.0, 52.5, 55.0, 57.5, 60.0, 62.5, 65.0, 67.5, 70.0, 72.5, 75.0, 77.5, 80.0, 82.5, 85.0, 87.5, 90.0, 92.5, 95.0, 97.5, 100.0, 102.5, 105.0, 107.5, 110.0, 112.5, 115.0, 117.5, 120.0, 122.5, 125.0})]]

In [12]:
for xyz in zip(*chains):
    print(*xyz)

OptionChain(exchange='NSE', underlyingConId=119653211, tradingClass='PCJEWELLER', multiplier='1', expirations={'20180726', '20180830', '20180927'}, strikes={260.0, 650.0, 140.0, 400.0, 20.0, 150.0, 280.0, 160.0, 550.0, 40.0, 300.0, 50.0, 180.0, 60.0, 700.0, 320.0, 450.0, 200.0, 80.0, 340.0, 600.0, 220.0, 350.0, 100.0, 750.0, 240.0, 500.0, 120.0, 250.0}) OptionChain(exchange='NSE', underlyingConId=80423707, tradingClass='JISLJALEQS', multiplier='1', expirations={'20180726', '20180830', '20180927'}, strikes={130.0, 135.0, 140.0, 145.0, 150.0, 155.0, 160.0, 35.0, 37.5, 165.0, 40.0, 42.5, 45.0, 47.5, 50.0, 52.5, 55.0, 57.5, 60.0, 62.5, 65.0, 67.5, 70.0, 72.5, 75.0, 77.5, 80.0, 82.5, 85.0, 87.5, 90.0, 92.5, 95.0, 97.5, 100.0, 102.5, 105.0, 107.5, 110.0, 112.5, 115.0, 117.5, 120.0, 122.5, 125.0})


In [247]:
def get_from_chains(get_this, chains):
    '''Gets the data from option chains
       Args:
          (get_this): string: <exchange> | <underlyingConId> | 
                              <tradingClass> | <multiplier> | 
                              <expirations> | <strikes>
           (chains): object: chains of options
           
       Returns:
          list of elements'''
 
    my_list = [getattr(i, get_this) for c in chains for i in c]
    
    if isinstance(my_list[0], set):
        my_list = [list(elem) for elem in my_list]
    
    return my_list

# scrip = get_from_chains('tradingClass', chains)
# strike = get_from_chains('strikes', chains)
# expiry = get_from_chains('expirations', chains)

scrip = [i.tradingClass for c in chains for i in c]
strike = list([i.strikes for c in chains for i in c])
expiry = [i.expirations for c in chains for i in c]

nest = [scrip, strike, expiry]
pd.DataFrame(nest, ['Scrip', 'Strike', 'Expiry']).T.ffill()


,Scrip,Strike,Expiry
0,PCJEWELLER,"{260.0, 650.0, 140.0, 400.0, 20.0, 150.0, 280....","{20180726, 20180830, 20180927}"
1,JISLJALEQS,"{130.0, 135.0, 140.0, 145.0, 150.0, 155.0, 160...","{20180726, 20180830, 20180927}"


In [ ]:
# instantiate stock contract class
scrip = Stock(symbol='WOCKPHARM', exchange='NSE' )
ib.qualifyContracts(scrip)

In [ ]:
# get the underlying's option chain with security definition parameter
chains = ib.reqSecDefOptParams(underlyingSymbol=scrip.symbol, futFopExchange = '', 
                               underlyingSecType=scrip.secType, underlyingConId=scrip.conId)

# put that in a dataframe
chains_df = util.df(chains)
chains_df.head()

In [ ]:
# make arrays for strikes and expiries
strikes = np.array(list(*chains_df.strikes))
expiries = np.array(list(*chains_df.expirations))

# function to get days to expiry
def get_dte(Expiry):
    '''Gives the expiry date
    Arg:
       (Expiry) = str of yyyymmdd format
    Returns:
       dte = int days to expiry'''
    exp_date = datetime.strptime(Expiry, '%Y%m%d')
    dte = (exp_date- datetime.now()).days
    return dte

dte = list(map(get_dte, expiries))

# function to get standard deviation
def get_stdev(contract, dte):
    '''Gets the Standard Deviation
    Args:
       (contract) = object: the qualified stock
       (dte)      = int: days to expiry
    Returns:
       standard deviation in days (int)
    '''
    sd_days = str(dte)+' D'
    bars = ib.reqHistoricalData(contract=scrip, endDateTime='', durationStr=sd_days, 
                                barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)
    sd = np.std([b.close for b in bars], ddof=1)
    return sd

stdev = list(map(lambda x, y: get_stdev(x, y), np.repeat(scrip, len(dte)), dte))

# equalize lengths of array for strikes+expiries+dte
a_strikes = np.repeat(strikes, len(expiries))
a_expiries = np.tile(expiries, len(strikes))   # tile to repeat expiries in blocks
a_dte = np.tile(dte, len(strikes))
a_stdev = np.tile(stdev, len(strikes))

# create a dataframe of Strike and Expiry
strikes_df = pd.DataFrame({'Strike': a_strikes, 'Expiry': a_expiries, 'DTE': a_dte, 'Stdev': a_stdev})

# insert a key and merge strikes and chains
strikes_df['key'] = 0
chains_df['key'] = 0

raw_chain = pd.merge(strikes_df, chains_df, on='key')
raw_chain = raw_chain.drop(['expirations', 'strikes', 'key'], axis=1)
raw_chain.head()

In [ ]:
contract = Option(symbol=scrip.symbol, exchange=scrip.exchange, 
                  lastTradeDateOrContractMonth='20180726', strike='640', right='P')
order = Order(action='SELL', orderType='MKT', totalQuantity=100.0)

In [ ]:
try:
    ib.whatIfOrder(contract, order)
except Exception as inst:
    print(inst.args)

In [ ]:
ib.whatIfOrder(contract, order)

In [ ]:
# Get the list of equity scrips
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1]          # It's the second table in the url

# Remove VIX and NIFTY from paisa
symbol = paisa.loc[~(paisa.Symbol.str.contains('VIX') | paisa.Symbol.str.contains('NIFTY'))].Symbol

paisa = paisa.loc[paisa.Symbol.isin(symbol), :]

# Make paisa's symbol compatible with IBKR symbol
paisa.loc[:, "Symbol"] = symbol.str.slice(0,9)

# Replace & for M&M, L&T, etc
mask = paisa.loc[paisa.Symbol.str.contains('&'), "Symbol"]
paisa.loc[paisa.Symbol.isin(mask), "Symbol"] = mask.str.replace('&','')

In [ ]:
paisa

In [ ]:
help(ib.events)

In [ ]:
[ticker] = ib.reqTickers(contract)

In [ ]:
ib.reqContractDetails(contract)

In [ ]:
ib.tickers()

# Original code of Ewald

In [ ]:
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

In [ ]:
# Get the ticker
[ticker] = ib.reqTickers(spx)
ticker

In [ ]:
spxValue = ticker.marketPrice()

In [ ]:
spxValue

In [ ]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)
util.df(chains)

In [ ]:
scrip = Stock('INTC', 'SMART', 'USD')
ib.qualifyContracts(scrip)

In [ ]:
scrip_chain = ib.reqSecDefOptParams(scrip.symbol, '', scrip.secType, scrip.conId)
util.df(scrip_chain)

In [ ]:
scrip_chain

In [ ]:
help(Contract.right)